In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from target_encoding import TargetEncoderClassifier, TargetEncoder
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from functools import reduce
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.model_selection import cross_val_score
import json
from functools import partial
from dsb2019.models.coeff import ThresholdClassifier

from dsb2019.models.tracking import track_experiment, track_submission_info
from dsb2019.data.validation import InstallationFold, cross_validate, quad_kappa
from dsb2019.visualization import session_browser
from dsb2019.data import DATA_DIR
from dsb2019.models import MODELS_DIR
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm
import hyperopt
from hyperopt import hp, fmin, Trials, tpe, STATUS_OK
tqdm.pandas()
pd.options.display.max_rows=999

In [3]:
train = pd.read_csv(DATA_DIR / 'raw/train.csv')
test = pd.read_csv(DATA_DIR / 'raw/test.csv')
train_labels = pd.read_csv(DATA_DIR / 'raw/train_labels.csv')
submission = pd.read_csv(DATA_DIR / 'raw/sample_submission.csv')

In [4]:
games = ['Scrub-A-Dub', 'All Star Sorting', 'Mushroom Sorter (Assessment)',
       'Air Show', 'Crystals Rule', 'Bird Measurer (Assessment)',
       'Dino Drink', 'Bubble Bath', 'Dino Dive', 'Chow Time',
       'Cauldron Filler (Assessment)', 'Pan Balance', 'Happy Camel',
       'Cart Balancer (Assessment)', 'Chest Sorter (Assessment)',
       'Leaf Leader']


def unwrap_event_data(df):
    unwrapped=pd.DataFrame(data=list(df.event_data.apply(json.loads).values))
    return pd.concat([unwrapped.reset_index(),df.reset_index()],axis=1)


def process_installations(train_labels, train, process_log):
    result = []
    train=train.sort_values("timestamp")
    installations = train.groupby("installation_id")
    for i, game_session, title, installation_id, accuracy_group in tqdm(train_labels[["game_session", "title", "installation_id", "accuracy_group"]].itertuples(), 
                                                              total=len(train_labels)):
        player_log = installations.get_group(installation_id).reset_index()
        log_length = player_log[(player_log.game_session==game_session) & (player_log.title==title)].index[0]
        player_log = player_log.iloc[:(log_length + 1)]
        player_log["accuracy_group"] = accuracy_group
        player_log["target_game_session"] = game_session
        features = process_log(player_log)
        features["installation_id"] = installation_id
        features["accuracy_group"] = accuracy_group
        result.append(features)
    return pd.DataFrame(data=result)
 
    
def process_log(df):
    assessment_title=df.title.iloc[-1]    

    history = df.iloc[:-1]
    history = history[history.type.isin(["Game", "Assessment"])].copy()

    def calculate_ratios(df):
        n_correct=df.correct_move.sum()
        n_incorrect=df.wrong_move.sum()
        ratio=n_correct/(n_correct+n_incorrect)
        return n_correct, n_incorrect, ratio
    
    def make_move_stats(df, title,n_lags=2):
        df=df.copy()
        if len(df):
            df = unwrap_event_data(df)
        if "correct" in df.columns:
            df["correct_move"] = df.correct == True
            df["wrong_move"] = df.correct == False
        else:
            df["correct_move"]=False
            df["wrong_move"]=False
        result = []
        result.extend(zip([f"n_correct {title}", f"n_incorrect {title}", f"global_ratio {title}"], calculate_ratios(df)))
        if n_lags:
            last_sessions = df.game_session.unique()[-n_lags:]
            for i in range(n_lags):
                if i < len(last_sessions): 
                    result.extend(zip([f"n_correct {title} {i}", f"n_incorrect {title} {i}",f"ratio {title} {i}"], 
                                      calculate_ratios(df[df.game_session==last_sessions[i]])))
                else:
                    result.extend(zip([f"n_correct {title} {i}", f"n_incorrect {title} {i}",f"ratio {title} {i}"], [None, None, None]))
        return {k: v for k, v in result}
    result = {"title": games.index(assessment_title)}
    for game in games:
        stats=history[history.title==game]
        stats=make_move_stats(stats, game)
        result.update(stats)
    return result

In [5]:
train_features = process_installations(train_labels, train, process_log)

100%|██████████| 17690/17690 [44:16<00:00,  6.66it/s] 


In [6]:
train_features=train_features.fillna(-1)

In [7]:
def lgb_quad_kappa(preds, true):
    true = true.get_label()
    #preds = preds.reshape((4, -1)).argmax(axis=0)
    preds = np.rint(preds)
    preds = np.maximum(0, preds)
    preds = np.minimum(3, preds)
    return "quad_kappa", quad_kappa(true, preds), True
    
    
def train_baseline(x_train,y_train, params=None):
    x_train_all, x_val_all,y_train_all,y_val_all = train_test_split(
        x_train,y_train,
        test_size=0.15,
        random_state=2019,
    )
    train_set = lgb.Dataset(x_train_all, y_train_all)
    val_set = lgb.Dataset(x_val_all, y_val_all)

#     params = {
#         'learning_rate': 0.01,
#         'bagging_fraction': 0.9,
#         'feature_fraction': 0.9,
#         'num_leaves': 14,
#         'lambda_l1': 0.1,
#         'lambda_l2': 1,
#         'metric': 'multiclass',
#         'objective': 'multiclass',
#         'num_classes': 4,
#         'random_state': 2019
#     }

    return lgb.train(params, train_set, num_boost_round=10000, early_stopping_rounds=2000, valid_sets=[val_set], verbose_eval=100)#,
                    #feval=lgb_quad_kappa)

def make_features(df):
    return df.drop(["installation_id", "accuracy_group"], axis=1), df.accuracy_group.values

def make_features_wrapper(train, test):
    def make_features(df):
        return df.drop(["installation_id", "accuracy_group"], axis=1), df.accuracy_group.values
    
    return make_features(train), make_features(test) 


def make_predictions(model,x_test_all,y_test):
    preds=model.predict(x_test_all)
    #preds = np.rint(preds)
    #preds = np.maximum(0, preds)
    #preds = np.minimum(3, preds)
    return preds,y_test

In [8]:
def process_test_installations(test):
    test = test.sort_values("timestamp")
    test=test.groupby("installation_id").progress_apply(process_log).reset_index()
    test.columns = ["installation_id", "features"]
    result = []
    for i, installation_id, feature in test.itertuples():
        result.append(feature)
        feature["installation_id"]=installation_id
    return pd.DataFrame(result).fillna(-1)
test_features=process_test_installations(test)

100%|██████████| 1000/1000 [01:34<00:00, 10.59it/s]


In [9]:
subtrain_installations=pd.Series(train_features.installation_id.unique()).sample(frac=1., random_state=2019)
subtrain_features=train_features[train_features.installation_id.isin(subtrain_installations.values)].copy()
def check_hyperparams(params):
    print(params)
    if "max_depth" in params:
        params["max_depth"] = int(params["max_depth"])
    if "num_leaves" in params:
        params["num_leaves"] = int(params["num_leaves"])

    train_baseline_with_params = partial(train_baseline, params=params)
    cv=InstallationFold(n_splits=3)
    predictions = cross_validate(subtrain_features, subtrain_features.accuracy_group, make_features_wrapper, train_baseline_with_params, make_predictions,
                                cv=cv)
    return {
        "loss": np.mean([mean_squared_error(true, pred) for pred, true in predictions]),
        "status": STATUS_OK,
        "params": params
    }


def tune(check_params, n_tries=25, n_learning_rate_tries=15, learning_rate=None, n_estimators=10_000):        
    if learning_rate is None:
        learning_rate_space = {
            'learning_rate': hp.loguniform("learning_rate", np.log(0.005), np.log(0.3)),
            'metric': 'rmse',
            'objective': 'rmse',
            #'num_classes': 4,
            'random_state': 2019,
            "n_estimators": n_estimators,

        }
        trials = Trials()
        result = fmin(check_params,
                      learning_rate_space, tpe.suggest, n_learning_rate_tries, trials=trials)
        print(result)
        learning_rate = round(trials.best_trial["result"]["params"]["learning_rate"], 3)

    param_space = {
        'metric': 'rmse',
        'objective': 'rmse',
        #'num_classes': 4,
        'lambda_l1': hp.uniform("lamba_l1", 1e-10, 1),
        'lambda_l2': hp.uniform("lambda_l2", 1e-10, 1),
        'random_state': 2019,
        "n_estimators": n_estimators,
        "learning_rate": learning_rate,
        "max_depth": hp.quniform("max_depth", 2, 16, 1),
        "num_leaves": hp.choice("num_leaves", [3, 7, 15, 31, 63, 127, 255, 511, 1023, 2047, 4095]),
        "subsample": hp.quniform("subsample", 0.01, 1, 0.01),
        "feature_fraction": hp.quniform("feature_fraction", 0.01, 1, 0.01),
    }

    trials = Trials()
    fmin(check_params,
         param_space, tpe.suggest, n_tries, trials=trials)
    best_params = trials.best_trial["result"]["params"]
    return best_params

In [10]:
best_params=tune(check_hyperparams, n_tries=100, n_learning_rate_tries=10)

{'learning_rate': 0.2250968304295338, 'metric': 'rmse', 'n_estimators': 10000, 'objective': 'rmse', 'random_state': 2019}
Training until validation scores don't improve for 2000 rounds
  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[100]	valid_0's rmse: 1.04243                       
[200]	valid_0's rmse: 1.05645                       
[300]	valid_0's rmse: 1.0659                        
[400]	valid_0's rmse: 1.07537                       
[500]	valid_0's rmse: 1.08137                       
[600]	valid_0's rmse: 1.0855                        
[700]	valid_0's rmse: 1.09225                       
[800]	valid_0's rmse: 1.09598                       
[900]	valid_0's rmse: 1.09906                       
[1000]	valid_0's rmse: 1.101                        
[1100]	valid_0's rmse: 1.10297                      
[1200]	valid_0's rmse: 1.10367                      
[1300]	valid_0's rmse: 1.10454                      
[1400]	valid_0's rmse: 1.10689                      
[1500]	valid_0's rmse: 1.10829                      
[1600]	valid_0's rmse: 1.1101                       
[1700]	valid_0's rmse: 1.11187                      
[1800]	valid_0's rmse: 1.11313                      
[1900]	valid_0's rmse: 1.11425                

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[100]	valid_0's rmse: 1.04319                                                
[200]	valid_0's rmse: 1.06199                                                
[300]	valid_0's rmse: 1.07065                                                
[400]	valid_0's rmse: 1.07932                                                
[500]	valid_0's rmse: 1.08373                                                
[600]	valid_0's rmse: 1.0865                                                 
[700]	valid_0's rmse: 1.09084                                                
[800]	valid_0's rmse: 1.09548                                                
[900]	valid_0's rmse: 1.09966                                                
[1000]	valid_0's rmse: 1.10427                                               
[1100]	valid_0's rmse: 1.10725                                               
[1200]	valid_0's rmse: 1.10804                                               
[1300]	valid_0's rmse: 1.11007                                  

...

[100]	valid_0's rmse: 1.03253                                                  
[200]	valid_0's rmse: 1.02831                                                  
[300]	valid_0's rmse: 1.03226                                                  
[400]	valid_0's rmse: 1.03695                                                  
[500]	valid_0's rmse: 1.04074                                                  
[600]	valid_0's rmse: 1.04567                                                  
[700]	valid_0's rmse: 1.0492                                                   
[800]	valid_0's rmse: 1.0513                                                   
[900]	valid_0's rmse: 1.05343                                                  
[1000]	valid_0's rmse: 1.05538                                                 
[1100]	valid_0's rmse: 1.05704                                                 
[1200]	valid_0's rmse: 1.0588                                                  
[1300]	valid_0's rmse: 1.06053          

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[100]	valid_0's rmse: 1.01698                                                    
[200]	valid_0's rmse: 1.00766                                                    
[300]	valid_0's rmse: 1.00771                                                    
[400]	valid_0's rmse: 1.0104                                                     
[500]	valid_0's rmse: 1.01268                                                    
[600]	valid_0's rmse: 1.01508                                                    
[700]	valid_0's rmse: 1.01769                                                    
[800]	valid_0's rmse: 1.01927                                                    
[900]	valid_0's rmse: 1.02191                                                    
[1000]	valid_0's rmse: 1.02458                                                   
[1100]	valid_0's rmse: 1.02629                                                   
[1200]	valid_0's rmse: 1.0292                                                    
[1300]	valid_0's

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[400]	valid_0's rmse: 1.12658                                                   
[500]	valid_0's rmse: 1.11765                                                   
[600]	valid_0's rmse: 1.11361                                                   
[700]	valid_0's rmse: 1.11056                                                   
[800]	valid_0's rmse: 1.10206                                                   
[900]	valid_0's rmse: 1.09794                                                   
[1000]	valid_0's rmse: 1.09631                                                  
[1100]	valid_0's rmse: 1.08968                                                  
[1200]	valid_0's rmse: 1.0882                                                   
[1300]	valid_0's rmse: 1.08566                                                  
[1400]	valid_0's rmse: 1.08346                                                  
[1500]	valid_0's rmse: 1.07973                                                  
[1600]	valid_0's rmse: 1.079

In [16]:
best_params

{'feature_fraction': 0.42,
 'lambda_l1': 0.7798520675772216,
 'lambda_l2': 0.8236580174630216,
 'learning_rate': 0.035,
 'max_depth': 13,
 'metric': 'rmse',
 'n_estimators': 10000,
 'num_leaves': 31,
 'objective': 'rmse',
 'random_state': 2019,
 'subsample': 0.61}

What was selected on 100% of the data

```
{'feature_fraction': 0.42,
 'lambda_l1': 0.7798520675772216,
 'lambda_l2': 0.8236580174630216,
 'learning_rate': 0.035,
 'max_depth': 13,
 'metric': 'rmse',
 'n_estimators': 10000,
 'num_leaves': 31,
 'objective': 'rmse',
 'random_state': 2019,
 'subsample': 0.61}```

In [12]:
# best_params={'feature_fraction': 0.58,
#  'lambda_l1': 0.45619796864269707,
#  'lambda_l2': 0.033257384218246686,
#  'learning_rate': 0.007,
#  'max_depth': 14,
#  'metric': 'multiclass',
#  'n_estimators': 10000,
#  'num_classes': 4,
#  'num_leaves': 31,
#  'objective': 'multiclass',
#  'random_state': 2019,
#  'subsample': 0.9500000000000001}

In [17]:
with open("../dsb2019/models/regression_baseline_rmse_params.json", "w") as f:
    json.dump(best_params, f)

In [18]:
baseline_model=train_baseline(train_features.drop(["installation_id", "accuracy_group"], axis=1), train_features.accuracy_group.values, 
               params=best_params)

Training until validation scores don't improve for 2000 rounds
[100]	valid_0's rmse: 1.01181
[200]	valid_0's rmse: 0.99135
[300]	valid_0's rmse: 0.990711
[400]	valid_0's rmse: 0.99123
[500]	valid_0's rmse: 0.992292
[600]	valid_0's rmse: 0.994112
[700]	valid_0's rmse: 0.996063
[800]	valid_0's rmse: 0.997587
[900]	valid_0's rmse: 0.999185
[1000]	valid_0's rmse: 1.00033
[1100]	valid_0's rmse: 1.00205
[1200]	valid_0's rmse: 1.00318
[1300]	valid_0's rmse: 1.00438
[1400]	valid_0's rmse: 1.00603
[1500]	valid_0's rmse: 1.00715
[1600]	valid_0's rmse: 1.00804
[1700]	valid_0's rmse: 1.00881
[1800]	valid_0's rmse: 1.00974
[1900]	valid_0's rmse: 1.01077
[2000]	valid_0's rmse: 1.01213
[2100]	valid_0's rmse: 1.01309
[2200]	valid_0's rmse: 1.01409
Early stopping, best iteration is:
[251]	valid_0's rmse: 0.990152


In [20]:
predictions = cross_validate(train_features, train_features.accuracy_group, make_features_wrapper, partial(train_baseline, params=best_params), 
                             make_predictions)
np.mean([mean_squared_error(true, pred) for pred, true in predictions]), [mean_squared_error(true, pred) for pred, true in predictions]

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 2000 rounds
[100]	valid_0's rmse: 1.01312
[200]	valid_0's rmse: 1.00325
[300]	valid_0's rmse: 1.00439
[400]	valid_0's rmse: 1.00642
[500]	valid_0's rmse: 1.00912
[600]	valid_0's rmse: 1.0122
[700]	valid_0's rmse: 1.01485
[800]	valid_0's rmse: 1.01674
[900]	valid_0's rmse: 1.01944
[1000]	valid_0's rmse: 1.02143
[1100]	valid_0's rmse: 1.02311
[1200]	valid_0's rmse: 1.02469
[1300]	valid_0's rmse: 1.02656
[1400]	valid_0's rmse: 1.02862
[1500]	valid_0's rmse: 1.03054
[1600]	valid_0's rmse: 1.0324
[1700]	valid_0's rmse: 1.03353
[1800]	valid_0's rmse: 1.03491
[1900]	valid_0's rmse: 1.03579
[2000]	valid_0's rmse: 1.03684
[2100]	valid_0's rmse: 1.03755
[2200]	valid_0's rmse: 1.03853
Early stopping, best iteration is:
[243]	valid_0's rmse: 1.00254
Training until validation scores don't improve for 2000 rounds
[100]	valid_0's rmse: 1.00288
[200]	valid_0's rmse: 0.991398
[300]	valid_0's rmse: 0.990939
[400]	valid_0's rmse: 0.992451
[500]	valid_0's

(1.0143329467268676,
 [0.9869776697581462,
  1.01201830738776,
  0.9660049527204333,
  1.0415864363637677,
  1.0650773674042306])

In [21]:
baseline_model.save_model(str(MODELS_DIR / "regression_baseline_rmse.lgb"))

In [22]:
features, target = make_features(train_features)
prediction=baseline_model.predict(features)
clf = ThresholdClassifier()
clf.fit(prediction, target)

100%|██████████| 1000/1000 [00:22<00:00, 45.20it/s, best loss: -0.6806236129755889]


ThresholdClassifier(n_iter=1000, random_state=2019)

In [23]:
clf.coef_

[1.1652705553319513, 1.7983061293534481, 2.2127169630913537]